In [1]:
""" Program to check that decomposition in QASM files is correct.
i.e. we take all Pauli strings from QASM file, exponentiate their sum with coeffitients 1 
and compare the output with given circuit.

As the result we know that the diagonalization is done correctly.
"""

import math
from icecream import ic
import qiskit_qasm2
from qiskit_qasm2 import load, CustomClassical
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
from scipy.linalg import expm
from numpy.linalg import norm, matrix_power
from functools import reduce
from itertools import product
from qiskit import QuantumCircuit
import qiskit.quantum_info as qi
from matplotlib.ticker import MaxNLocator
import time

from os import listdir
from os.path import isfile, join

PAULIS = {'I': np.eye(2, dtype='complex'),
          'X': np.array([[0, 1], [1, 0]], dtype='complex'),
          'Y': np.array([[0, -1j], [1j, 0]]),
          'Z': np.diag(np.array([1, -1], dtype='complex'))}


## for higher order the coefficients are of order 10^24/10^28
B_sign = {3: (np.array([-1,0,1]),2),
          5: (np.array([1,-8,0,8,-1]),12),
          7: (np.array([-1,9,-45,0,45,-9,1]),60),
          9: (np.array([3,-32,168,-672,0,672,-168,32,-3]),840),
          11: (np.array([-2,25,-150,600,-2100,0,2100,-600,150,-25,2]),2520)}

def make_B(N, coefs, cbase):
    """ make matrix B s.t. L = -BB.T with correct zero boundary conditions.
    N - size of B.
    coefs - list of coefficients.
    cbase - divider.
    
    """
    B = np.zeros((N,N))
    l = len(coefs)//2
    diags = list(range(-l,l+1))
    for i,val in zip(diags,coefs):
        B += val*np.eye(N,k=i)

    ### Boundary connditions
    ## left side
    B[0,:] = np.zeros_like(B[0,:])
    B[:,0] = np.zeros_like(B[:,0])
    
    for i in range(l):
        coef_temp = coefs[-(i+1):]
        B[l-i,:len(coef_temp)] = B[l-i,:len(coef_temp)] - coef_temp
        # print(l-i, coef_temp)
    B[:,0] = B[:,0]*np.sqrt(2)

    ## right side
    B[-1,:] = np.zeros_like(B[-1,:])
    B[:,-1] = np.zeros_like(B[:,-1])
    for i in range(l):
        coef_temp = coefs[:(i+1)]
        B[-(l-i)-1,-len(coef_temp):] = B[-(l-i)-1,-len(coef_temp):] - coef_temp
        # print(-(l-i)-1, coef_temp)
    B[:,-1] = B[:,-1]*np.sqrt(2)
    
    # return B
    return B/cbase

In [4]:
def W(gr, wl, cb):
    alpha = 1
    weights_from_func_W.append(alpha)
    return 2*alpha

customs = [
    CustomClassical("w", 3, W),
]

In [9]:
"""Parameters"""

# ic.disable()

acc_order = 6 # to take files from SAMPLEOUT-{acc_order}

NQ = 4
nq = NQ - 1

"""Reading QASM files"""

mypath = 'SAMPLEOUT-{}'.format(acc_order)
file_names = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f)) and '-{}_{}_'.format(acc_order,NQ) in f]
dict_sets = dict(zip(range(len(file_names)),file_names))

H_rec = 0
for filename in file_names:
    """Read Pauli strings and coefficients (signs) from file """
    p_str_list = []
    p_str_list_diag = []

    with open(filename) as f:
        counter = 0 # to count lines starting with //
        for line in f:
            line = line.strip()
            if 'Differential coefficients' in line:
                counter = 1

            if counter == 1: # diagonal part (Rz)
                if 'rz(' in line:
                    temp = line.split()[2] # //XIII,+ZIII
                    p_str_d = temp[-NQ-1:] # e.g. +IZIZI
                    p_str = temp[2:NQ+2] # e.g. XYXYX

                    p_str_list.append(p_str)
                    p_str_list_diag.append(p_str_d)
    
    """Read circuit from QASM file """
    
    weights_from_func_W = []
    circuit_l = load(filename, custom_classical=customs).reverse_bits()
    matrix_l = qi.Operator(circuit_l).data

    ic(filename, p_str_list, p_str_list_diag, weights_from_func_W)

    """Compare results"""
    
    H_group = 0 # reconstructed Hamiltonian (sum of alpha_j P_j)
    alpha_test = 1
    expm_prod = np.eye(2**NQ) # One step in Trotter formula (Product of expm(-i t alpha_j P_j)
    sum_error_coef = 0
    
    for p_str, p_str_d, wwH in zip(p_str_list,p_str_list_diag, weights_from_func_W):
        sign = p_str_d[0]
        if p_str_d[0] == '+':
            sign = 1
        else:
            sign = -1
        P = reduce(np.kron, [PAULIS[s] for s in p_str])
        H_group += sign*alpha_test*P

    exp_H_group = expm(-1j*H_group)
    ic(norm(exp_H_group - matrix_l))

ic.enable()

ic| filename: 'SAMPLEOUT-6/hdecomp-6_4_15_GD.qasm'
    p_str_list: ['YXXY', 'XYXY', 'XYYX', 'YXYX', 'XXYY', 'YYYY', 'YYXX', 'XXXX']
    p_str_list_diag: ['+ZIII', '+IZII', '+IZZI', '+ZIZI', '-IIZZ', '+ZZZZ', '-ZZIZ', '+IIIZ']
    weights_from_func_W: [1, 1, 1, 1, 1, 1, 1, 1]
ic| norm(exp_H_group - matrix_l): 4.293151472727913e-15
ic| filename: 'SAMPLEOUT-6/hdecomp-6_4_13_GD.qasm'
    p_str_list: ['YXIY', 'XYIY', 'XYZY', 'YXZY', 'XXZX', 'YYZX', 'YYIX', 'XXIX']
    p_str_list_diag: ['+ZIII', '+IZII', '+IZZI', '+ZIZI', '+IIZZ', '-ZZZZ', '-ZZIZ', '+IIIZ']
    weights_from_func_W: [1, 1, 1, 1, 1, 1, 1, 1]
ic| norm(exp_H_group - matrix_l): 2.7235843614255604e-15
ic| filename: 'SAMPLEOUT-6/hdecomp-6_4_10_GD.qasm'
    p_str_list: ['YIYI', 'XZXI', 'YZYI', 'XIXI', 'XIXZ', 'YZYZ', 'XZXZ', 'YIYZ']
    p_str_list_diag: ['+ZIII', '+IZII', '+ZZZI', '+IIZI', '+IIZZ', '+ZZZZ', '+IZIZ', '+ZIIZ']
    weights_from_func_W: [1, 1, 1, 1, 1, 1, 1, 1]
ic| norm(exp_H_group - matrix_l): 2.7235843614255604e-15
ic